# Grafo de condiciones -> ingredientes -> productos

In [10]:
import json
import pandas as pd
import networkx as nx
import pickle

In [11]:
cleaned_products_csv_path = "cleaning/cleaned_products.csv"
ingredients_mappings_csv_path = "cleaning/ingredients_mappings.csv"
standardized_ingredients_fichas_csv_path = "cleaning/ingredientes_fichas_standardized.json"

In [12]:
products_df = pd.read_csv(cleaned_products_csv_path)
products_df.head(2)

,title,price,info,composition,usage,precautions,brand,image,description,url,limpiar,tratar,proteger,composition_list,composition_list_standard,id
0,fotoprotector isdin fusión water pediatrics spf50,s/ 124.90,"['todo tipo de piel, incluso piel atópica y/o ...","vitamina e, provitamina b5, extracto de siempr...",aplica fusion water magic pediatrics spf 50 ge...,uso externo. mantener el producto fuera del al...,isdin,https://dcuk1cxrnzjkh.cloudfront.net/imagespro...,fotoprotector isdin fusión water pediatrics sp...,https://inkafarma.pe/producto/fotoprotector-is...,False,False,True,"['vitamina e', 'provitamina b5', 'extracto de ...","['tocoferol', 'extracto de siempreviva', 'vita...",P0000
1,crema anti-irritación la roche posay lipikar 4...,s/ 176.90,"['calma la piel inmediatamente.', 'con su efic...","aqua posae filiformis , microresyl: devuelve e...",aplicar sobre la piel limpia con masaje circul...,mantener fuera del alcance de los niños.,la roche-posay,https://dcuk1cxrnzjkh.cloudfront.net/imagespro...,crema anti-irritación la roche posay lipikar b...,https://inkafarma.pe/producto/lipikar-baume-ap...,False,True,False,"['aqua posae filiformis', 'microresyl', 'resta...","['manteca de karité', 'glicerina', 'niacinamid...",P0001


In [13]:
products_df["composition_list_standard"] = products_df.composition_list_standard.str.strip("[]").str.split(",").apply(lambda x: [i.strip().strip("'") for i in x])
products_df.head(2)

,title,price,info,composition,usage,precautions,brand,image,description,url,limpiar,tratar,proteger,composition_list,composition_list_standard,id
0,fotoprotector isdin fusión water pediatrics spf50,s/ 124.90,"['todo tipo de piel, incluso piel atópica y/o ...","vitamina e, provitamina b5, extracto de siempr...",aplica fusion water magic pediatrics spf 50 ge...,uso externo. mantener el producto fuera del al...,isdin,https://dcuk1cxrnzjkh.cloudfront.net/imagespro...,fotoprotector isdin fusión water pediatrics sp...,https://inkafarma.pe/producto/fotoprotector-is...,False,False,True,"['vitamina e', 'provitamina b5', 'extracto de ...","[tocoferol, extracto de siempreviva, vitamina b5]",P0000
1,crema anti-irritación la roche posay lipikar 4...,s/ 176.90,"['calma la piel inmediatamente.', 'con su efic...","aqua posae filiformis , microresyl: devuelve e...",aplicar sobre la piel limpia con masaje circul...,mantener fuera del alcance de los niños.,la roche-posay,https://dcuk1cxrnzjkh.cloudfront.net/imagespro...,crema anti-irritación la roche posay lipikar b...,https://inkafarma.pe/producto/lipikar-baume-ap...,False,True,False,"['aqua posae filiformis', 'microresyl', 'resta...","[manteca de karité, glicerina, niacinamida, ag...",P0001


In [14]:
ingredients_df = pd.read_csv(ingredients_mappings_csv_path, encoding="utf-8", header=None, names=["name", "clean_name", "id"])
ingredients_df.head(2)

,name,clean_name,id
0,extracto de siempreviva,extracto de siempreviva,I0000
1,microresyl,microresyl,I0001


In [15]:
with open(standardized_ingredients_fichas_csv_path, "r", encoding="utf-8") as f:
    ingredientes_fichas = json.load(f)
ingredientes_fichas[0:2]

[{'chemical': 'ácido cítrico',
  'conditions': ['stains', 'dark_spot', 'skinredness']},
 {'chemical': 'ácido glicólico',
  'conditions': ['acne_scar',
   'wrinkle',
   'stains',
   'dark_spot',
   'white_head',
   'papules']}]

In [16]:
# nodos del grafo
skin_conditions = ["stains", "dark_spot", "skinredness", "acne_scar",
                   "wrinkle", "papules", "white_head", "black_head",
                   "pustules", "nodules", "dark_circle", "eye_bag",
                   "vascular", "freckle"] # hyperkeratosis
ingredients = ingredients_df[ingredients_df.clean_name != "INVÁLIDO"].clean_name.unique().tolist()
products = products_df.title.unique().tolist()

In [17]:
ingredients_df[ingredients_df.clean_name == "INVÁLIDO"]

,name,clean_name,id
55,protege,INVÁLIDO,I0052
56,de amplio espectro,INVÁLIDO,I0052
57,cortos​,INVÁLIDO,I0052
60,con ph 5.5,INVÁLIDO,I0052
62,marcas,INVÁLIDO,I0052
...,...,...,...
774,oxido de hierro para reforzar la protección fr...,INVÁLIDO,I0052
775,base limpiadora sin jabón,INVÁLIDO,I0052
776,mejor conocidos como ahs,INVÁLIDO,I0052
777,es apto para todo tipo de piel,INVÁLIDO,I0052


In [18]:
# Agregar los nodos al grafo
graph = nx.Graph()
graph.add_nodes_from(skin_conditions, type="skin_condition")
graph.add_nodes_from(ingredients, type="ingredient")
graph.add_nodes_from(products, type="product")

In [19]:
# Agregar aristas al grafo
# Primero de skin_conditions a ingredientes
for item in ingredientes_fichas:
    ing = item["chemical"]
    for cond in item["conditions"]:
        # Check if the condition node exists
        if cond not in graph:
            print(f"Warning: Node '{cond}' does not exist. Adding it as a 'skin_condition'.")
            graph.add_node(cond, type="skin_condition")
        
        # Check if the ingredient node exists
        if ing not in graph:
            print(f"Warning: Node '{ing}' does not exist. Adding it as an 'ingredient'.")
            graph.add_node(ing, type="ingredient")
        
        # Add the edge
        graph.add_edge(cond, ing, type="skin_condition_to_ingredient")

In [20]:
# Ahora de ingredientes a productos
for index, row in products_df.iterrows():
    product = row["title"]
    for ingredient in row["composition_list_standard"]:
        # Check if the ingredient node exists
        if ingredient == "": # a veces no hay ingredientes y la lista se vuelve [], resultando en un ingrediente vacío ""
            print(f"Warning: Empty ingredient in product '{product}'. Skipping this ingredient.")
            continue
        
        if ingredient not in graph:
            print(f"Warning: Node '{ingredient}' does not exist. Adding it as an 'ingredient'.")
            graph.add_node(ingredient, type="ingredient")
        
        # Check if the product node exists
        if product not in graph:
            print(f"Warning: Node '{product}' does not exist. Adding it as a 'product'.")
            graph.add_node(product, type="product")
        
        # Add the edge
        graph.add_edge(ingredient, product, type="ingredient_to_product")

In [21]:
def get_products_for_condition(G, condition):
    products = set()
    for ingredient in G.neighbors(condition):
        print(f"Ingredient: {ingredient}")  # Print the ingredient
        for product in G.neighbors(ingredient):
            if G.nodes[product]["type"] == "product":
                products.add(product)
    return products

In [22]:
recommended_products = get_products_for_condition(graph, "stains")
for prod in recommended_products:
    row = products_df[products_df.title == prod]
    print(row["title"].values[0], ":", row["composition_list_standard"].values[0])

Ingredient: ácido cítrico
Ingredient: ácido glicólico
eucerin serum antimanchas dermopure triple effect 40ml : ['pantenol', 'glucosilrutina', 'almidón de tapioca', 'resorcinol amido tiazólico isobutilo', 'pantolactona', 'ácido cítrico', 'ácido salicílico', 'edta trisódico', 'glicerina', 'carbómero', 'agua', 'isoquercitrina', 'alcohol denaturado', 'extracto de raíz de glycyrrhiza inflata', 'fragancia', 'hidróxido de sodio', 'glicol decileno', 'copolímero de acrililo dimetiltaurato de amonio-VP']
crema gel eucerin hyaluron filler ultra light : ['pantenol', 'hialuronato de sodio', 'escualano', 'pantolactona', 'alcohol denaturado', 'fenoxietanol', 'alcohol cetílico', 'ácido cítrico', 'hidroxihidrocinamato de pentaeritritilo tetra-di-t-butilo', 'carragenina', 'ácido láctico', 'tocoferol', 'goma de gellan', 'fosfato de distarch', 'lisina lauroil', 'alcohol behenílico', 'isononanoato de cetearyl', 'goma xantana', 'palmitato de isopropilo', 'glicerina', 'agua', 'fragancia']
aceite de ducha euc

In [23]:
with open("skincare_graph.pkl", "wb") as f:
    pickle.dump(graph, f)

In [26]:
# Contar el grado de los nodos de tipo "skin_condition"
skin_condition_degrees = {node: graph.degree(node) for node in graph.nodes if graph.nodes[node].get('type') == 'skin_condition'}

# Mostrar los grados
for condition, degree in skin_condition_degrees.items():
    print(f"Condición: {condition}, Grado: {degree}")

Condición: stains, Grado: 2
Condición: dark_spot, Grado: 7
Condición: skinredness, Grado: 11
Condición: acne_scar, Grado: 5
Condición: wrinkle, Grado: 8
Condición: papules, Grado: 7
Condición: white_head, Grado: 4
Condición: black_head, Grado: 1
Condición: pustules, Grado: 1
Condición: nodules, Grado: 1
Condición: dark_circle, Grado: 1
Condición: eye_bag, Grado: 1
Condición: vascular, Grado: 1
Condición: freckle, Grado: 0
